In [ ]:
Importing the libraries

# to read the dataframe
import pandas as pd
Reading the dataset as a dataframe

# Reading dataframes
customers_df = pd.read_csv('customers.csv')
orders_df = pd.read_csv('orders.csv')
products_df = pd.read_csv('products.csv')
sales_df = pd.read_csv('sales.csv')

customers_df.head()
customer_id	customer_name	gender	age	home_address	zip_code	city	state	country
0	1	Leanna Busson	Female	30	8606 Victoria TerraceSuite 560	5464	Johnstonhaven	Northern Territory	Australia
1	2	Zabrina Harrowsmith	Female	69	8327 Kirlin SummitApt. 461	8223	New Zacharyfort	South Australia	Australia
2	3	Shina Dullaghan	Male	59	269 Gemma SummitSuite 109	5661	Aliburgh	Australian Capital Territory	Australia
3	4	Hewet McVitie	Male	67	743 Bailey GroveSuite 141	1729	South Justinhaven	Queensland	Australia
4	5	Rubia Ashleigh	Male	30	48 Hyatt ManorSuite 375	4032	Griffithsshire	Queensland	Australia

orders_df.head()
order_id	customer_id	payment	order_date	delivery_date
0	1	64	30811	2021-8-30	2021-09-24
1	2	473	50490	2021-2-3	2021-02-13
2	3	774	46763	2021-10-8	2021-11-03
3	4	433	39782	2021-5-6	2021-05-19
4	5	441	14719	2021-3-23	2021-03-24

products_df.head()
product_ID	product_type	product_name	size	colour	price	quantity	description
0	0	Shirt	Oxford Cloth	XS	red	114	66	A red coloured, XS sized, Oxford Cloth Shirt
1	1	Shirt	Oxford Cloth	S	red	114	53	A red coloured, S sized, Oxford Cloth Shirt
2	2	Shirt	Oxford Cloth	M	red	114	54	A red coloured, M sized, Oxford Cloth Shirt
3	3	Shirt	Oxford Cloth	L	red	114	69	A red coloured, L sized, Oxford Cloth Shirt
4	4	Shirt	Oxford Cloth	XL	red	114	47	A red coloured, XL sized, Oxford Cloth Shirt

sales_df.head()
sales_id	order_id	product_id	price_per_unit	quantity	total_price
0	0	1	218	106	2	212
1	1	1	481	118	1	118
2	2	1	2	96	3	288
3	3	1	1002	106	2	212
4	4	1	691	113	3	339
Discovering the dataset

#show the number of cells in customers dataframe
print("customers dataframe size: ", customers_df.size)
​
#show the number of records (rows) in customers dataframe
print("number of customers: ", len(customers_df))
customers dataframe size:  9000
number of customers:  1000

customers_df.dtypes
customer_id       int64
customer_name    object
gender           object
age               int64
home_address     object
zip_code          int64
city             object
state            object
country          object
dtype: object

orders_df.dtypes
order_id          int64
customer_id       int64
payment           int64
order_date       object
delivery_date    object
dtype: object

products_df.dtypes
product_ID       int64
product_type    object
product_name    object
size            object
colour          object
price            int64
quantity         int64
description     object
dtype: object

sales_df.dtypes
sales_id          int64
order_id          int64
product_id        int64
price_per_unit    int64
quantity          int64
total_price       int64
dtype: object
Activities
Let's enjoy wrangling data in these dataframes.
1. Calculate the total sales for each product category.

products_sales_df = pd.merge(products_df, sales_df, left_on='product_ID', right_on='product_id', how='left')
total_sales_by_category = products_sales_df.groupby('product_type')['total_price'].sum().reset_index(name='total_price')
total_sales_by_category['total_price'] = total_sales_by_category['total_price'].fillna(0)
​

total_sales_by_category
product_type	total_price
0	Jacket	357026.0
1	Shirt	333600.0
2	Trousers	341174.0
2. Find the total number of orders and the average order value for each customer.

comb = pd.merge(customers_df, orders_df, on='customer_id')
customer_analysis = comb.groupby('customer_id')[['order_id','payment']].agg({'order_id':'count','payment':'mean'}).rename(columns={'order_id':'total_orders','payment':'average_payment'})

customer_analysis
total_orders	average_payment
customer_id
1	3	23463.0
7	1	48935.0
10	1	45626.0
11	1	41952.0
12	1	39451.0
...	...	...
994	1	58159.0
995	1	58829.0
996	2	26014.0
998	4	25318.5
1000	1	35622.0
617 rows × 2 columns
3. Determine the total quantity sold and remaining quantity for each product.

comb = pd.merge(products_df,sales_df,how='left',left_on='product_ID', right_on='product_id', suffixes=('_product','_sales'))
total_quantity_sold = comb.groupby('product_ID')['quantity_sales'].sum()
product_inventory = pd.merge(products_df,total_quantity_sold,how='left',on='product_ID').fillna(0)
product_inventory['remaining_quantity'] = product_inventory['quantity']-product_inventory['quantity_sales']

comb.head()
product_ID	product_type	product_name	size	colour	price	quantity_product	description	sales_id	order_id	product_id	price_per_unit	quantity_sales	total_price
0	0	Shirt	Oxford Cloth	XS	red	114	66	A red coloured, XS sized, Oxford Cloth Shirt	NaN	NaN	NaN	NaN	NaN	NaN
1	1	Shirt	Oxford Cloth	S	red	114	53	A red coloured, S sized, Oxford Cloth Shirt	22.0	5.0	1.0	96.0	3.0	288.0
2	1	Shirt	Oxford Cloth	S	red	114	53	A red coloured, S sized, Oxford Cloth Shirt	1121.0	231.0	1.0	96.0	1.0	96.0
3	1	Shirt	Oxford Cloth	S	red	114	53	A red coloured, S sized, Oxford Cloth Shirt	1741.0	353.0	1.0	96.0	3.0	288.0
4	1	Shirt	Oxford Cloth	S	red	114	53	A red coloured, S sized, Oxford Cloth Shirt	3201.0	644.0	1.0	96.0	2.0	192.0
4. Determine the total sales and average order value for each city.

comb = customers_df.merge(orders_df, how='left', on='customer_id').merge(sales_df,on='order_id',how='left')
grouped = comb.groupby('city')[['total_price','payment']].agg({'total_price':'sum','payment':'mean'}).fillna(0).rename(columns={'total_price':'total_price_in_sales','payment':'average_order_payment'})
geographic_analysis = grouped
5. Identify returning customers.

comb = customers_df.merge(orders_df, on='customer_id')
returning_customers = comb[comb.duplicated(subset='customer_id')]
returning_customers = returning_customers[['customer_id','customer_name']].drop_duplicates()

returning_customers
customer_id	customer_name
1	1	Leanna Busson
9	15	Avril Rossiter
12	18	Chan Duchesne
15	20	Gigi Kalaher
17	21	Seana Hearons
...	...	...
983	986	Theodosia Postins
985	987	Grady Wipper
988	989	Sauncho Boosey
994	996	Elvira Sarfati
996	998	Rebecka Fabler
263 rows × 2 columns
6. Determine the average order value by gender.

comb = customers_df.merge(orders_df, on='customer_id').merge(sales_df, on='order_id')
avg_order_value_by_gender = comb.groupby('gender')[['total_price','order_id']].agg({'total_price':'mean','order_id':'count'}).reset_index().rename(columns={'total_price':'average_total_price','order_id':'number_of_orders'})
​
​

avg_order_value_by_gender
gender	average_total_price	number_of_orders
0	Female	204.917963	2572
1	Male	207.887562	2428
7. Determine the total orders price per age group.

# Create age groups
customers_age_group_df = customers_df.copy()
bins = [0, 25, 40, 60, 100]
labels = ['<25', '25-40', '40-60', '60+']
customers_age_group_df['age_group'] = pd.cut(customers_age_group_df['age'], bins=bins, labels=labels)
comb = customers_age_group_df.merge(orders_df, on='customer_id').merge(sales_df,on='order_id')
sales_by_age_group = comb.groupby('age_group')['total_price'].sum()
8. Determine the most sold colors.

comb = products_df.merge(sales_df, left_on='product_ID', right_on='product_id', suffixes=('_product','_sales'))
most_sold_colors = comb.groupby('colour')['quantity_sales'].sum().reset_index().sort_values(by='quantity_sales', ascending=False)
9. Determine the top selling products in each city.

comb = customers_df.merge(orders_df,on='customer_id').merge(sales_df, on='order_id').merge(products_df, left_on='product_id',right_on='product_ID',suffixes=('_sales','_product'))
products_sales_orders_merged_df = comb.groupby(['city','product_name']).agg(total_quantity_sold=('quantity_sales','sum')).reset_index()
top_selling_products_by_city = products_sales_orders_merged_df.sort_values(['city', 'total_quantity_sold'], ascending=[True, False])
top_selling_products_by_city = top_selling_products_by_city.groupby('city').head(1)

top_selling_products_by_city
city	product_name	total_quantity_sold
1	Abbeyshire	Chambray	4
14	Abbottburgh	Oxford Cloth	4
21	Abbottbury	Pullover	5
30	Abigailshire	Peacoat	5
42	Adamsside	Pullover	5
...	...	...	...
4284	Youngview	Drawstring	3
4288	Zacharyborough	Cuban Collar	3
4303	Zacharybury	Tracksuit Bottoms	4
4305	Zacshire	Cardigan	2
4308	Zacville	Denim	3
604 rows × 3 columns
10. Determine the name of the male and female customers who paid the most.

comb = customers_df.merge(orders_df, on='customer_id')
male = comb.loc[comb.gender=='Male'].groupby(['customer_name','gender'])['payment'].agg('sum').reset_index()
female = comb.loc[comb.gender=='Female'].groupby(['customer_name','gender'])['payment'].sum().reset_index()
​
top_female = female[['customer_name','gender','payment']][female.payment==female.payment.max()].rename(columns={'payment':'paid_amount'})
top_male = male[['customer_name','gender','payment']][male.payment==male.payment.max()].rename(columns={'payment':'paid_amount'})
​

top_female.index=[410]
top_male.index=[347]
11. Determine the number of products in the 5 fastest orders in terms of delivery time.

order_sales_merged_df = pd.merge(orders_df, sales_df, on='order_id')
order_sales_merged_df['delivery_time'] = pd.to_datetime(order_sales_merged_df['delivery_date']) - pd.to_datetime(order_sales_merged_df['order_date'])
order_sales_merged_df['delivery_time'] = order_sales_merged_df['delivery_time'].dt.days
fastest_orders = order_sales_merged_df.groupby('order_id').agg(product_count=('order_id', 'size'), delivery_time=('delivery_time', 'first')).reset_index()
fastest_orders = fastest_orders.sort_values('delivery_time', ascending=True).head(5)
​
​
12. Determine the average delivery time by state.

cus_orders_merged_df = pd.merge(orders_df, customers_df, on='customer_id', how='right')
cus_orders_merged_df['order_date'] = cus_orders_merged_df['order_date'].fillna(pd.to_datetime('today').date())
cus_orders_merged_df['delivery_date'] = cus_orders_merged_df['delivery_date'].fillna(pd.to_datetime('today').date())
cus_orders_merged_df['delivery_time'] = (pd.to_datetime(cus_orders_merged_df['delivery_date']) - pd.to_datetime(cus_orders_merged_df['order_date'])).dt.days
avg_delivery_time_by_state = cus_orders_merged_df.groupby('state')['delivery_time'].mean().reset_index()
​
avg_delivery_time_by_state = avg_delivery_time_by_state.rename(columns={'delivery_time': 'average_delivery_time_in_days'})
​
13. Determine the total sales per price range.

# Create price ranges based on the price column in the products_df
products_price_ranges_df = products_df.copy()
price_ranges = [70, 80, 90, 100, 110, 120]
price_labels = ['$70-$80', '$81-$90', '$91-$100', '$101-$110', '$111-$120']
products_price_ranges_df['price_range'] = pd.cut(products_price_ranges_df['price'], bins=price_ranges, labels=price_labels)
comb = products_price_ranges_df.merge(sales_df, left_on='product_ID',right_on='product_id', suffixes=('_product','_sale'))
price_range_analysis = comb.groupby('price_range').agg(total_sales=('total_price','sum')).reset_index().sort_values(by='price_range')

price_range_analysis
price_range	total_sales
0	70−80	0
1	81−90	62040
2	91−100	326001
3	101−110	204778
4	111−120	438981